In [17]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import joblib
warnings.filterwarnings('ignore')

# Point FastF1 to your cache folder
fastf1.Cache.enable_cache('../data/fastf1_cache')

print("✅ FastF1 ready")
print(f"FastF1 version: {fastf1.__version__}")

Matplotlib is building the font cache; this may take a moment.


✅ FastF1 ready
FastF1 version: 3.8.1


In [19]:
fastf1.Cache.enable_cache('../data/fastf1_cache')

RESULTS_PATH = '../data/race_results_2022_2025.csv'
QUALI_PATH = '../data/qualifying_2022_2025.csv'

# ============================================================
# WHAT WE STILL NEED
# Results:    2023 races 19-23, 2024 races 2-24, 2025 races 1-24
# Qualifying: ALL races (2022 r1-22, 2023 r1-23, 2024 r1-24, 2025 r1-24)
# ============================================================

collection_plan = {
    2025: {'results': (1, 24), 'quali': (1, 24)},
}


def get_race_results(year, race_number):
    try:
        session = fastf1.get_session(year, race_number, 'R')
        session.load(telemetry=False, weather=False, messages=False)
        
        results = session.results[['DriverNumber', 'Abbreviation',
                                    'TeamName', 'GridPosition',
                                    'Position', 'Status', 'Points']].copy()
        results['Year'] = year
        results['RaceNumber'] = race_number
        results['EventName'] = session.event['EventName']
        results['CircuitName'] = session.event['Location']
        
        return results
    except Exception as e:
        print(f"❌ {e}")
        return None


def get_qualifying_results(year, race_number):
    try:
        session = fastf1.get_session(year, race_number, 'Q')
        session.load(telemetry=False, weather=False, messages=False)
        
        quali = session.results[['DriverNumber', 'Abbreviation',
                                  'Q1', 'Q2', 'Q3', 'Position']].copy()
        quali.columns = ['DriverNumber', 'Abbreviation',
                         'Q1Time', 'Q2Time', 'Q3Time', 'QualiPosition']
        quali['Year'] = year
        quali['RaceNumber'] = race_number
        
        def best_time(row):
            for t in [row['Q3Time'], row['Q2Time'], row['Q1Time']]:
                if pd.notna(t):
                    return t.total_seconds()
            return None
        
        quali['BestQualiTime'] = quali.apply(best_time, axis=1)
        return quali
    
    except Exception as e:
        print(f"❌ {e}")
        return None


def save_data(new_df, path, dedup_cols):
    """Save incrementally — append to existing file and drop duplicates"""
    if new_df is None:
        return
    
    if os.path.exists(path):
        existing = pd.read_csv(path)
        combined = pd.concat([existing, new_df], ignore_index=True)
        combined = combined.drop_duplicates(subset=dedup_cols)
    else:
        combined = new_df
    
    combined.to_csv(path, index=False)


# ============================================================
# MAIN COLLECTION LOOP
# ============================================================
api_calls = 0

for year, plan in collection_plan.items():
    print(f"\n{'='*50}")
    print(f"📅 {year}")
    print(f"{'='*50}")
    
    # Figure out all unique race numbers we need to touch
    race_nums_results = set(range(plan['results'][0], plan['results'][1] + 1)) if plan['results'] else set()
    race_nums_quali   = set(range(plan['quali'][0],   plan['quali'][1] + 1))   if plan['quali']   else set()
    all_race_nums     = sorted(race_nums_results | race_nums_quali)
    
    for race_num in all_race_nums:
        print(f"\n  Race {race_num}:", end=' ')
        
        # Pause every 50 calls to respect rate limit
        if api_calls > 0 and api_calls % 50 == 0:
            print(f"\n  ⏸️  Pausing 60s to respect rate limit ({api_calls} calls made)...")
            time.sleep(60)
        
        # Collect results if needed for this race
        if race_num in race_nums_results:
            print("results...", end=' ')
            result = get_race_results(year, race_num)
            save_data(
                result,
                RESULTS_PATH,
                dedup_cols=['Year', 'RaceNumber', 'Abbreviation']
            )
            api_calls += 1
            time.sleep(2)
            print("✅" if result is not None else "❌", end=' ')
        
        # Collect qualifying if needed for this race
        if race_num in race_nums_quali:
            print("quali...", end=' ')
            quali = get_qualifying_results(year, race_num)
            save_data(
                quali,
                QUALI_PATH,
                dedup_cols=['Year', 'RaceNumber', 'Abbreviation']
            )
            api_calls += 1
            time.sleep(2)
            print("✅" if quali is not None else "❌", end=' ')

# ============================================================
# FINAL SUMMARY
# ============================================================
print(f"\n\n🏁 Collection complete! Total API calls: {api_calls}")

results_final = pd.read_csv(RESULTS_PATH)
print(f"\n✅ Results — {len(results_final)} rows")
print(results_final.groupby('Year')['RaceNumber'].max().to_string())

if os.path.exists(QUALI_PATH):
    quali_final = pd.read_csv(QUALI_PATH)
    print(f"\n✅ Qualifying — {len(quali_final)} rows")
    print(quali_final.groupby('Year')['RaceNumber'].max().to_string())


📅 2025

  Race 1: results... 

core           INFO 	Loading data for Australian Grand Prix - Race [v3.8.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22

✅ quali... 

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '4'
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '1', '63', '22', '23', '16', '44', '10', '55', '6', '14', '18', '7', '5', '12', '27', '30', '31', '87']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.8.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


✅ 
  Race 2: results... 

...

✅ 
  Race 24: results... 

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

✅ quali... 

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

✅ 

🏁 Collection complete! Total API calls: 48

✅ Results — 1838 rows
Year
2022    22
2023    22
2024    24
2025    24

✅ Qualifying — 1839 rows
Year
2022    22
2023    22
2024    24
2025    24


In [34]:
# Make sure every race has both results AND qualifying
results = pd.read_csv('../data/race_results_2022_2025.csv')
quali = pd.read_csv('../data/qualifying_2022_2025.csv')

# Merge them to check alignment
merged = results.merge(
    quali[['Abbreviation', 'Year', 'RaceNumber', 'QualiPosition', 'BestQualiTime']],
    on=['Abbreviation', 'Year', 'RaceNumber'],
    how='left',
    indicator=True
)

# Check for any races where quali is missing
missing_quali = merged[merged['_merge'] == 'left_only']

if len(missing_quali) > 0:
    print(f"⚠️  {len(missing_quali)} rows missing qualifying data:")
    print(missing_quali[['Year', 'RaceNumber', 'Abbreviation', 'EventName']].head(10))
else:
    print("✅ Perfect! Every race result has matching qualifying data")
    print(f"\nTotal complete records: {len(merged)}")

✅ Perfect! Every race result has matching qualifying data

Total complete records: 1838
